In [1]:
# !pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2
import numpy as np  

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'videoData/deadlift/hips.avi'
EXPORT_PATH = 'annotations/deadlift/hips.csv'
MODEL_PATH = 'models/deadlift/hips.pkl'

1.1 Save Video

In [7]:
cap = cv2.VideoCapture(0)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter(VIDEO_PATH, cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

cv2.VideoCapture()

while cap.isOpened():
    ret,frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    try:
        cv2.imshow('Press', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()


2. Capture Landmars and Export to CSV

In [4]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [5]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [6]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist() #Converted to a list
        # print(keypoints)
        keypoints.insert(0,action)
        
        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f,delimiter = ',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [8]:
cap = cv2.VideoCapture(VIDEO_PATH) #name of the video
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        k = cv2.waitKey(1)
        if k== ord('a'):
            export_landmark(results,'wide')
        elif k== ord('s'):
            export_landmark(results,'neutral')
        elif k== ord('d'):
            export_landmark(results,'narrow')
        
        cv2.imshow('Hips Webcam Feed',image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


### 3. Train Custom Model Using Scikit Learn

3.1 Read in Collected Data and Process

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_csv(EXPORT_PATH)

In [11]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,narrow,0.519627,0.315274,-0.349930,0.999956,0.525882,0.302391,-0.328807,0.999877,0.530171,...,0.177596,0.401604,0.525392,0.967436,-0.019264,0.846338,0.497061,0.970687,0.024036,0.762010
1,narrow,0.519725,0.313397,-0.382662,0.999962,0.525550,0.300616,-0.362022,0.999889,0.529610,...,0.174333,0.429354,0.524904,0.979328,-0.000884,0.868965,0.498972,0.971087,0.016058,0.799572
2,narrow,0.519667,0.313377,-0.385201,0.999962,0.525522,0.300607,-0.364939,0.999888,0.529591,...,0.178707,0.424636,0.524945,0.979316,-0.000289,0.867492,0.498042,0.971924,0.022384,0.796672
3,narrow,0.519296,0.313499,-0.380858,0.999962,0.525231,0.300649,-0.360507,0.999888,0.529402,...,0.178862,0.423080,0.525135,0.979573,0.001741,0.868360,0.497856,0.972852,0.024166,0.797516
4,narrow,0.519269,0.313501,-0.386208,0.999963,0.525203,0.300648,-0.365588,0.999891,0.529367,...,0.181589,0.426639,0.525271,0.979573,0.001058,0.872485,0.497753,0.973233,0.027320,0.803004


In [12]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
772,wide,0.502345,0.315919,-0.397795,0.999822,0.506938,0.301860,-0.377395,0.999552,0.509866,...,0.213899,0.395040,0.518955,0.975995,0.053352,0.698535,0.440479,0.980794,0.061090,0.693057
773,wide,0.505542,0.312623,-0.403747,0.999843,0.510757,0.298137,-0.383179,0.999603,0.514178,...,0.163252,0.401710,0.522557,0.988117,0.016634,0.718382,0.438034,0.993043,-0.005180,0.711135
774,wide,0.505165,0.312579,-0.413394,0.999849,0.510610,0.297218,-0.393689,0.999617,0.514098,...,0.187482,0.408799,0.520194,0.987237,0.025863,0.731929,0.438428,0.992353,0.019357,0.722638
775,wide,0.506576,0.311007,-0.420167,0.999860,0.512310,0.295627,-0.400326,0.999645,0.515869,...,0.205501,0.423185,0.516301,0.987424,0.026108,0.747305,0.444195,0.993749,0.041694,0.735583
776,wide,0.508932,0.309313,-0.380408,0.999866,0.514796,0.294485,-0.358492,0.999663,0.518211,...,0.150321,0.414280,0.517466,1.000265,-0.008485,0.740046,0.448422,1.001459,-0.015045,0.725023


In [13]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33


In [14]:
X = df.drop('class', axis=1) #Features
Y = df['class'] #Target value

In [15]:
X_train, X_test, y_train,y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1234)

In [20]:
Y

0      narrow
1      narrow
2      narrow
3      narrow
4      narrow
        ...  
772      wide
773      wide
774      wide
775      wide
776      wide
Name: class, Length: 777, dtype: object

3.2 Train Machine Learning Classification Model

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [22]:
pipelines = {
    'lr':make_pipeline(StandardScaler(),LogisticRegression()),
    'rc':make_pipeline(StandardScaler(),RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [23]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

d:\Shashaank\Clg\MiniProjs\Sem7\AI_Trainer\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

3.3 Evaluate and Serialize Model


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [26]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
            precision_score(y_test.values,yhat,average='weighted'),
            recall_score(y_test.values,yhat,average='weighted'))

lr 0.9957264957264957 0.9957893413775766 0.9957264957264957
rc 1.0 1.0 1.0
rf 0.9957264957264957 0.9957893413775766 0.9957264957264957
gb 1.0 1.0 1.0


In [27]:
with open(MODEL_PATH,'wb') as f:  #dumping the model
    pickle.dump(fit_models['gb'],f)

4. Make Detections with Model

In [28]:
with open(MODEL_PATH,'rb') as f:
    model = pickle.load(f)

In [30]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
#Initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detections
        results = pose.process(image)

        #Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                )

        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)

            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = 'up'
                counter += 1
                print(current_stage,counter)

            #Get status box
            cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

            #Display Class
            cv2.putText(image,'CLASS',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Probability
            cv2.putText(image,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            #Display Counter
            cv2.putText(image,'COUNTER',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(counter),(175,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        except Exception as e:
            print(e)
            pass
        
        cv2.imshow('Raw Webcam Feed', image)

        if(cv2.waitKey(10) & 0xFF == ord('q')):
            break

cap.release()
cv2.destroyAllWindows()


wide [5.12432615e-07 4.07654520e-06 9.99995411e-01]
wide [5.41742734e-07 4.07290927e-06 9.99995385e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.32802706e-07 4.07290931e-06 9.99995394e-01]
wide [5.35878131e-07 4.09641881e-06 9.99995368e-01]
wide [5.36356376e-07 3.57882958e-06 9.99995885e-01]
wide [5.38069958e-07 3.59026342e-06 9.99995872e-01]
wide [5.38069958e-07 3.59026342e-06 9.99995872e-01]
wide [5.37156738e-07 3.67049706e-06 9.99995792e-01]
wide [5.23399149e-07 3.59026347e-06 9.99995886e-01]
wide [5.23399149e-07 3.59026347e-06 9.99995886e-01]
wide [5.43320367e-07 4.07654507e-06 9.99995380e-01]
wide [5.43320367e-07 4.07654507e-06 9.99995380e-01]
wide [5.43320367e-07 4.07654507e-06 9.99995380e-01]
wide [5.4283